In [3]:
!pip install torch torchvision torchaudio

  Using cached torch-2.7.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.22.1-cp312-cp312-win_amd64.whl.metadata (6.1 kB)
  Using cached torchaudio-2.7.1-cp312-cp312-win_amd64.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
Using cached torch-2.7.1-cp312-cp312-win_amd64.whl (216.1 MB)
Using cached torchvision-0.22.1-cp312-cp312-win_amd64.whl (1.7 MB)
Using cached torchaudio-2.7.1-cp312-cp312-win_amd64.whl (2.5 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)


In [7]:
# IMPORTING LIBRARIES ---------------------------------------------

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

import numpy as np
import os
from tqdm import tqdm

#DEFINING FEATURES - RGB, Resize, Normalize ------------------------

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

#LOAD DATASET -------------------------------------------------------

train_data = ImageFolder(root='data/train', transform=transform)
test_data = ImageFolder(root='data/test', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [9]:
resnet = models.resnet18(pretrained=True)
resnet.eval()

# Remove the final classification layer
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])


C:\Users\nader\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\nader\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\nader/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [02:14<00:00, 348kB/s]


In [15]:
def extract_features(dataloader):
    features = []
    labels = []

    with torch.no_grad():
        for inputs, targets in tqdm(dataloader):
            outputs = feature_extractor(inputs)
            outputs = outputs.view(outputs.size(0), -1)  # Flatten
            features.append(outputs)
            labels.append(targets)

    X = torch.cat(features).numpy()
    y = torch.cat(labels).numpy()
    return X, y


In [17]:
X_train, y_train = extract_features(train_loader)
X_test, y_test = extract_features(test_loader)

np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)
np.save('X_test.npy', X_test)
np.save('y_test.npy', y_test)


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:16<00:00,  1.80s/it]


In [21]:
cd C:\Users\nader\project\plant-classifier


C:\Users\nader\project\plant-classifier
